# Trajectory Partition

In [1]:
import math
import pandas as pd

In [ ]:
def find_longer_shorter(line1, line2):
    line1_length = segment_length(line1)
    line2_length = segment_length(line2)
    if line1_length > line2_length:
        return line1, line2
    else:
        return line2, line1


def segment_length(line):
    length = ((line[0][0] - line[1][0])**2 + (line[0][1] - line[1][1])**2)**0.5
    return length


def get_unit_vector(line):
    line_length = segment_length(line)
    if line_length == 0:
        return [0, 0]
    unit_vector = [line[1][0], line[1][1]]
    unit_vector[0] = (unit_vector[0] - line[0][0]) / line_length
    unit_vector[1] = (unit_vector[1] - line[0][1]) / line_length
    return unit_vector


def projection_distance(point, line):
    diff_x = point[0] - line[0][0]
    diff_y = point[1] - line[0][1]
    unit_vector = get_unit_vector(line)
    projection_distance =  abs(diff_x * unit_vector[1] - diff_y * unit_vector[0])
    return projection_distance


def perpendicular_distance(line1, line2):
    longer, shorter = find_longer_shorter(line1, line2)
    distance1 = projection_distance(shorter[0], longer)
    distance2 = projection_distance(shorter[1], longer)
    if distance1 == 0 and distance2 == 0:
        return 0
    perpendicular_distance = (distance1**2 + distance2**2) / (distance1 + distance2)
    return round(perpendicular_distance, 10)


def angle_distance(line1, line2):
    longer, shorter = find_longer_shorter(line1, line2)
    unit_vector_shorter = get_unit_vector(shorter)
    unit_vector_longer = get_unit_vector(longer)
    sin_coefficient = unit_vector_shorter[0] * unit_vector_longer[1] - unit_vector_shorter[1] * unit_vector_longer[0]
    angle_distance = sin_coefficient * segment_length(shorter)
    return abs(angle_distance)


def cost(trajectory, start_index, current_index, partition):
    if partition:
        segment = [trajectory[start_index], trajectory[current_index]]
        seg_length = segment_length(segment)
        if seg_length != 0:
            LH = math.log2(segment_length(segment))
        else:
            LH = 0
        LDH_p = 0
        LDH_a = 0
        i = 0
        while start_index + i < current_index:
            index1 = start_index + i
            index2 = start_index + i + 1
            line = [trajectory[index1], trajectory[index2]]
            distance_p = perpendicular_distance(line, segment)
            distance_a = angle_distance(line, segment)
            LDH_p += distance_p
            LDH_a += distance_a
            i += 1
        if LDH_p == 0 and LDH_a == 0:
            LDH = 0
        elif LDH_p == 0 and LDH_a != 0:
            LDH = math.log2(LDH_a)
        elif LDH_p != 0 and LDH_a == 0:
            LDH = math.log2(LDH_p)
        else:
            LDH = math.log2(LDH_p) + math.log2(LDH_a)
        cost = LH + LDH
    else:
        cost = 0
        i = 0
        while start_index + i < current_index:
            index1 = start_index + i
            index2 = start_index + i + 1
            line = [trajectory[index1], trajectory[index2]]
            line_length = segment_length(line)
            if line_length != 0:
                cost += math.log2(line_length)
            i += 1
    return cost

def trajectory_partition(trajectory):
    cp = []
    cp_index = []
    cp.append(trajectory[0])
    cp_index.append(0)
    start_index = 0
    length = 1
    c = 0
    while start_index + length < len(trajectory):
        current_index = start_index + length
        cost_par = cost(trajectory, start_index, current_index, True)
        cost_non = cost(trajectory, start_index, current_index, False)
        if cost_par > cost_non:
            cp.append(trajectory[current_index-1])
            cp_index.append(current_index-1)
            start_index = current_index-1
            length = 1
        else:
            length += 1
        c += 1
    cp.append(trajectory[-1])
    cp_index.append(len(trajectory)-1)
    return cp, cp_index

In [ ]:
def preprocess():
    data_dir = '../data/porto-data'
    data_name = "small-porto"
    trajectories = pd.read_csv("{}/{}.csv".format(data_dir, data_name), header=0, index_col="TRIP_ID")
    total_traj_num = len(trajectories)

    shortest, longest = 20, 1200
    processed_trajectories = []
    for i, (idx, traj) in enumerate(trajectories.iterrows()):
        if i % 10000 == 0:
            print("Complete: {}; Total: {}".format(i, total_traj_num))
        polyline = eval(traj["POLYLINE"])
        coordinates = []
        if shortest <= len(polyline) <= longest:
            for lng, lat in polyline:
                coordinates.append([lng, lat])
            processed_trajectories.append(coordinates)

    fout = open("{}/processed_{}.csv".format(data_dir, data_name), 'w')
    fout.write("POLYLINE\n")
    for traj in processed_trajectories:
        fout.write('"' + str(traj) + '"' + "\n")
    fout.close()

    print("finished")

In [ ]:
def partition():
    trajectories = pd.read_csv("../data/porto-data/processed_small-porto.csv")
    all_cp_index = []
    for i, (idx, traj) in enumerate(trajectories.iterrows()):
        if i % 10000 == 0:
            print("progress:", i, "/", len(trajectories))
        traj_coord = eval(traj["POLYLINE"])
        _, cp_index = trajectory_partition(traj_coord)
        all_cp_index.append(cp_index)
        fout = open("../data/porto-data/processed_small_porto_with_cp.csv", 'w')

    fout.write("POLYLINE, CP_INDEX\n")
    for i, (idx, traj) in enumerate(trajectories.iterrows()):
        fout.write('"' + str(traj["POLYLINE"]) + '",')
        fout.write('"' + str(traj["CP_INDEX"]) + '"\n')
    fout.close()

    print("finished")

# Data Preprocess

In [37]:
def height2lat(height):
    return height / 110.574


def width2lng(width):
    return width / 111.320 / 0.99974


def in_boundary(lat, lng, b):
    return b['min_lng'] < lng < b['max_lng'] and b['min_lat'] < lat < b['max_lat']


def data_preprocess():
    grid_height, grid_width = 0.1, 0.1
    boundary = {'min_lat': 41.140092, 'max_lat': 41.185969, 'min_lng': -8.690261, 'max_lng': -8.549155}

    trajectories = pd.read_csv("../data/porto-data/processed_small_porto_with_cp.csv")

    lat_size, lng_size = height2lat(grid_height), width2lng(grid_width)

    lat_grid_num = int((boundary['max_lat'] - boundary['min_lat']) / lat_size) + 1
    lng_grid_num = int((boundary['max_lng'] - boundary['min_lng']) / lng_size) + 1

    trajectories = pd.read_csv("../data/porto-data/processed_small_porto_with_cp.csv")
    processed_trajectories = []
    processed_subtrajectories = []

    total_traj_num = len(trajectories)
    for i, (idx, traj) in enumerate(trajectories.iterrows()):

        if i % 10000 == 0:
            print("Complete: {}; Total: {}".format(i, total_traj_num))

        grid_seq = []
        valid = True
        polyline = eval(traj["POLYLINE"])
        for lng, lat in polyline:
            if in_boundary(lat, lng, boundary):
                grid_i = int((lat - boundary['min_lat']) / lat_size)
                grid_j = int((lng - boundary['min_lng']) / lng_size)
                grid_seq.append((grid_i, grid_j))
            else:
                valid = False
                break
        grid_seq_subtraj = []
        if valid:
            processed_trajectories.append(grid_seq)
            cp_index = eval(traj[" CP_INDEX"])
            for index in range(len(cp_index)-1):
                cp_start_index = cp_index[index]
                cp_end_index = cp_index[index+1]
                subtraj_gird = grid_seq[cp_start_index: cp_end_index]
                processed_subtrajectories.append(subtraj_gird)
    print("Grid size:", (lat_grid_num, lng_grid_num))
    print("Total valid trajectory number:", len(processed_trajectories))
    print("Total subtrajectory number:", len(processed_subtrajectories))

    fout = open("../data/porto-data/processed_small_porto_subtrajectory.csv", 'w')
    for traj in processed_subtrajectories:
        fout.write("[")
        for i, j in traj[:-1]:
            fout.write("%s, " % str(i * lng_grid_num + j))
        fout.write("%s]\n" % str(traj[-1][0] * lng_grid_num + traj[-1][1]))
    fout.close()

    print("finished")

In [ ]:
preprocess()

In [ ]:
partition()

In [38]:
data_preprocess()

Complete: 0; Total: 89011
Complete: 10000; Total: 89011
Complete: 20000; Total: 89011
Complete: 30000; Total: 89011
Complete: 40000; Total: 89011
Complete: 50000; Total: 89011
Complete: 60000; Total: 89011
Complete: 70000; Total: 89011
Complete: 80000; Total: 89011
Grid size: (51, 158)
Total valid trajectory number: 66247
Total subtrajectory number: 1257246
